In [1]:
# first of all, let's check that avro tools work

!java -jar /usr/bin/avro.jar

Version 1.12.0
 of Apache Avro
Copyright 2010-2015 The Apache Software Foundation

This product includes software developed at
The Apache Software Foundation (https://www.apache.org/).
----------------
Available tools:
    canonical  Converts an Avro Schema to its canonical form
          cat  Extracts samples from files
      compile  Generates Java code for the given schema.
       concat  Concatenates avro files without re-compressing.
        count  Counts the records in avro files or folders
  fingerprint  Returns the fingerprint for the schemas.
   fragtojson  Renders a binary-encoded Avro datum as JSON.
     fromjson  Reads JSON records and writes an Avro data file.
     fromtext  Imports a text file into an avro data file.
      getmeta  Prints out the metadata of an Avro data file.
    getschema  Prints out schema of an Avro data file.
          idl  Generates a JSON schema or protocol from an Avro IDL file
 idl2schemata  Extract JSON schemata of the types from an Avro IDL fil

In [1]:
# check that Spark client works

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .config(
            "spark.sql.extensions",
            "org.projectnessie.spark.extensions.NessieSparkSessionExtensions, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
        )
        .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .config("spark.sql.catalog.iceberg.type", "rest")
        .config("spark.sql.catalog.iceberg.uri", "http://nessie:19120/iceberg/main/")
        .config("spark.sql.catalog.iceberg.ref", "main")
        .config("spark.sql.catalog.iceberg.cache-enabled", False)
        .getOrCreate()
)
spark.sparkContext.setLogLevel('ERROR')

spark.sql("""
    SHOW TABLES FROM iceberg.default
""").show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|  default|bank_transfers|      false|
+---------+--------------+-----------+



In [ ]:
# check that Trino client works

from trino.dbapi import connect

trino_connection = connect(
    host="trino",
    port=8080,
    user="trino",
)
trino = trino_connection.cursor()
trino.execute("SELECT * FROM system.runtime.nodes")

rows = trino.fetchall()
print(rows)

In [ ]:
# check that Python client works

from pyiceberg.catalog.hive import HiveCatalog
from pyiceberg.exceptions import NoSuchTableError

catalog = HiveCatalog(name="iceberg", uri="thrift://hivemetastore:9083")

try:
    catalog.load_table("default.sample")
except NoSuchTableError as e:
    print(e)